<a href="https://colab.research.google.com/github/adasamonte/AgenticAI-NewsPrompt/blob/main/FinalProject_Cabrera_Samonte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas nltk beautifulsoup4 sentence-transformers faiss-cpu gensim scikit-learn transformers emoji

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import faiss
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Load dataset and reduce to 500-2000 rows
df = pd.read_csv('/content/bbc_news.csv')
df = df.sample(n=700, random_state=42).reset_index(drop=True)
df.head()


,title,pubDate,guid,link,description
0,US's only Palestinian-American Congresswoman c...,"Wed, 08 Nov 2023 10:19:15 GMT",https://www.bbc.co.uk/news/world-us-canada-673...,https://www.bbc.co.uk/news/world-us-canada-673...,"Michigan Democrat defends pro-Palestinian ""riv..."
1,Murdered driver's family demand help for couriers,"Sun, 25 Feb 2024 21:52:55 GMT",https://www.bbc.co.uk/news/uk-wales-67726081,https://www.bbc.co.uk/news/uk-wales-67726081,Mark Lang was killed by a man who was stealing...
2,Cleared pony owner criticises 'trial by social...,"Fri, 25 Aug 2023 18:20:35 GMT",https://www.bbc.co.uk/news/uk-england-leiceste...,https://www.bbc.co.uk/news/uk-england-leiceste...,"Sarah Moulds also criticises the RSPCA, saying..."
3,Wrexham: Can football fame make City of Cultur...,"Sat, 06 May 2023 07:52:28 GMT",https://www.bbc.co.uk/news/uk-wales-65494230,https://www.bbc.co.uk/news/uk-wales-65494230?a...,Backers who want Wrexham to win City of Cultur...
4,Gaza ceasefire talks intensify in Cairo,"Sat, 04 May 2024 19:53:46 GMT",https://www.bbc.co.uk/news/world-middle-east-6...,https://www.bbc.co.uk/news/world-middle-east-6...,Hamas said it was sending negotiators to talks...


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

texts = df['description'].tolist()
embeddings = model.encode(texts, show_progress_bar=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
sia = SentimentIntensityAnalyzer()

df['sentiment'] = df['description'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['sentiment_label'] = df['sentiment'].apply(
    lambda x: 'positive' if x >= 0.05 else ('negative' if x <= -0.05 else 'neutral')
)

df[['description', 'sentiment', 'sentiment_label']].head()


,description,sentiment,sentiment_label
0,"Michigan Democrat defends pro-Palestinian ""riv...",0.0000,neutral
1,Mark Lang was killed by a man who was stealing...,-0.8481,negative
2,"Sarah Moulds also criticises the RSPCA, saying...",-0.6124,negative
3,Backers who want Wrexham to win City of Cultur...,0.9246,positive
4,Hamas said it was sending negotiators to talks...,0.6486,positive


In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words]

texts_tokenized = df['description'].apply(preprocess)

dictionary = corpora.Dictionary(texts_tokenized)
corpus = [dictionary.doc2bow(text) for text in texts_tokenized]

lda = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Display Topics
topics = lda.print_topics(num_words=5)
for topic in topics:
    print(topic)

# Coherence
coherence_model_lda = CoherenceModel(model=lda, texts=texts_tokenized, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_score}")

(0, '0.006*"people" + 0.006*"says" + 0.004*"bbc" + 0.004*"family" + 0.004*"week"')
(1, '0.008*"says" + 0.006*"world" + 0.005*"people" + 0.005*"bbc" + 0.005*"england"')
(2, '0.007*"says" + 0.006*"new" + 0.005*"bbc" + 0.005*"world" + 0.003*"cup"')
(3, '0.008*"says" + 0.007*"first" + 0.005*"win" + 0.005*"time" + 0.004*"watch"')
(4, '0.009*"says" + 0.005*"one" + 0.004*"years" + 0.004*"england" + 0.004*"home"')
Coherence Score: 0.5390537036041214


In [ ]:
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def rag_query(query, k=5, sentiment_filter=None):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)

    retrieved = [texts[i] for i in I[0]]
    if sentiment_filter:
        filtered = []
        for i in I[0]:
            if df.iloc[i]['sentiment_label'] == sentiment_filter:
                filtered.append(df.iloc[i]['description'])
        retrieved = filtered or retrieved

    context = " ".join(retrieved)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = qa_pipeline(prompt, max_length=256)
    return response[0]['generated_text']


In [ ]:
# Manually labeled sample (20 for Demonstraiton)
manual_labels = df.sample(20, random_state=123).reset_index(drop=True)

# Display for manual inspection
for i, row in manual_labels.iterrows():
    print(f"[{i}] {row['description']}")

manual_labels['true_sentiment'] = [
    'neutral', 'positive', 'negative', 'neutral', 'positive',
    'negative', 'positive', 'neutral', 'negative', 'positive',
    'neutral', 'neutral', 'negative', 'positive', 'negative',
    'positive', 'neutral', 'negative', 'neutral', 'positive'
]

#Evaluate
y_true = manual_labels['true_sentiment']
y_pred = manual_labels['sentiment_label']

# Inspect unique values and data types
print("Unique values in y_true:", y_true.unique())
print("Data type of y_true:", y_true.dtype)
print("Unique values in y_pred:", y_pred.unique())
print("Data type of y_pred:", y_pred.dtype)

# Print value counts to check distribution
print("Value counts for y_true:\n", y_true.value_counts())
print("Value counts for y_pred:\n", y_pred.value_counts())

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_true_encoded = le.fit_transform(y_true)
y_pred_encoded = le.transform(y_pred)


print("Accuracy:", accuracy_score(y_true_encoded, y_pred_encoded))
print("Precision:", precision_score(y_true_encoded, y_pred_encoded, average='weighted'))
print("Recall:", recall_score(y_true_encoded, y_pred_encoded, average='weighted'))
print("F1 Score:", f1_score(y_true_encoded, y_pred_encoded, average='weighted'))

[0] We asked climate experts from around the world what they want to see agreed at COP26 this week.
[1] Jordan McSweeney beat Zara Aleena to death as she walked home from a night out in east London.
[2] Russia's foreign minister tells the BBC that the West is pressing UN officials to amplify fake news.
[3] BBC Arabic's Adnan El-Bursh speaks to residents in a destroyed neighbourhood in Gaza.
[4] Bryn Perry, from Pembrokeshire, recycles leftover whey from his cheese-making to produce the spirit.
[5] Increased intelligence sharing after the 9/11 attacks may have made leaks more likely.
[6] The iconic carmaker's rebrand has certainly caught people's attention, but will it pay off?
[7] The former president opened up the USSR but failed to prevent its collapse in 1991.
[8] The late Russian opposition leader's team says Volkov was assaulted with a hammer and pepper sprayed.
[9] Police have been urged to crack down on the practice, which retailers say is an increasing trend.
[10] An image of 4

In [ ]:
def precision_at_k(retrieved, relevant, k):
    return len(set(retrieved[:k]) & set(relevant)) / k

def recall_at_k(retrieved, relevant, k):
    return len(set(retrieved[:k]) & set(relevant)) / len(relevant)

def reciprocal_rank(retrieved, relevant):
    for rank, doc_id in enumerate(retrieved, start=1):
        if doc_id in relevant:
            return 1.0 / rank
    return 0.0


queries = [
    "How is the war affecting international trade?",
    "What is the global political response to the Ukraine crisis?",
    "What are the economic consequences of the conflict?",
    "How are refugees impacted by the war in Ukraine?",
    "What actions are world leaders taking in response?",
    "What are the sanctions imposed by NATO?",
    "How has the energy market been influenced?",
    "What humanitarian efforts are ongoing?",
    "How are global food prices being affected?",
    "What role is the UN playing in resolving the crisis?"
]

# Ground truth indices ( should be based on manual review)
ground_truth = [
    [2, 4, 19, 67, 500],
    [5, 15, 42, 112],
    [4, 19, 310, 455, 102],
    [592, 164, 12, 71, 48],
    [0, 680, 13, 21, 597],
    [500, 15, 9, 517],
    [232, 225, 521, 38],
    [141, 122, 48, 88, 99],
    [141, 674, 67, 105],
    [18, 33, 71, 258, 9]
]

doc_ids = df.index.tolist()

retrieved_docs = []
for query in queries:
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), 5)
    retrieved_ids = [doc_ids[i] for i in I[0]]
    retrieved_docs.append(retrieved_ids)

total_rr = 0.0

for i, retrieved in enumerate(retrieved_docs):
    p = precision_at_k(retrieved, ground_truth[i], 5)
    r = recall_at_k(retrieved, ground_truth[i], 5)
    rr = reciprocal_rank(retrieved, ground_truth[i])
    total_rr += rr
    print(f"Query {i+1} - P@5: {p:.2f}, R@5: {r:.2f}, RR: {rr:.2f}")

mrr = total_rr / len(queries)
print(f"\nMean Reciprocal Rank (MRR): {mrr:.4f}")

for i, retrieved in enumerate(retrieved_docs):
    print(f"Query {i+1}: Retrieved = {retrieved}, Ground Truth = {ground_truth[i]}")
    print("Intersection:", set(retrieved).intersection(set(ground_truth[i])))

Query 1 - P@5: 0.20, R@5: 0.20, RR: 0.20
Query 2 - P@5: 0.00, R@5: 0.00, RR: 0.00
Query 3 - P@5: 0.40, R@5: 0.40, RR: 0.33
Query 4 - P@5: 0.60, R@5: 0.60, RR: 1.00
Query 5 - P@5: 0.40, R@5: 0.40, RR: 0.50
Query 6 - P@5: 0.60, R@5: 0.75, RR: 1.00
Query 7 - P@5: 0.80, R@5: 1.00, RR: 1.00
Query 8 - P@5: 0.20, R@5: 0.20, RR: 0.50
Query 9 - P@5: 0.40, R@5: 0.50, RR: 1.00
Query 10 - P@5: 0.40, R@5: 0.40, RR: 0.25

Mean Reciprocal Rank (MRR): 0.5783
Query 1: Retrieved = [517, 127, 646, 455, 500], Ground Truth = [2, 4, 19, 67, 500]
Intersection: {500}
Query 2: Retrieved = [164, 148, 71, 127, 53], Ground Truth = [5, 15, 42, 112]
Intersection: set()
Query 3: Retrieved = [127, 646, 310, 455, 517], Ground Truth = [4, 19, 310, 455, 102]
Intersection: {310, 455}
Query 4: Retrieved = [592, 164, 321, 71, 327], Ground Truth = [592, 164, 12, 71, 48]
Intersection: {592, 164, 71}
Query 5: Retrieved = [51, 680, 127, 521, 597], Ground Truth = [0, 680, 13, 21, 597]
Intersection: {680, 597}
Query 6: Retrieved

In [ ]:
def rag_query_verbose(query, k=5, sentiment_filter=None):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)

    retrieved = [texts[i] for i in I[0]]
    if sentiment_filter:
        filtered = []
        for i in I[0]:
            if df.iloc[i]['sentiment_label'] == sentiment_filter:
                filtered.append(df.iloc[i]['description'])
        retrieved = filtered or retrieved

    context = " ".join(retrieved)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = qa_pipeline(prompt, max_length=256)

    print("=== Context ===\n", context[:1000], "\n...\n")
    print("=== Generated Answer ===\n", response[0]['generated_text'])
    return response[0]['generated_text']

# Test one for review
rag_query_verbose("What are the economic consequences of the conflict?")


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


=== Context ===
 UK concern over the conflict in the Middle East escalating even further makes headlines on Saturday.   The BBC hears from three people whose opposition to the war cost them their employment. The deal comes as the value of the cryptocurrency has plunged. The father and daughter in disagreement over the war that killed a beloved brother and son. How does a European country stay neutral when war rages in Europe? Many Swiss are rethinking their position. 
...

=== Generated Answer ===
 The value of the cryptocurrency has plunged


'The value of the cryptocurrency has plunged'

In [ ]:
test_queries = [
    "How are refugees impacted by the war in Ukraine?",
    "What humanitarian efforts are ongoing in Ukraine?",
    "What actions are world leaders taking in response to the conflict?",
    "How is global trade disrupted by the Ukraine crisis?",
    "What role is the United Nations playing in resolving the war?",
    "What are the economic consequences of the conflict?",
    "What are the sanctions imposed by NATO?",
    "How has the energy market been influenced?",
    "What humanitarian efforts are ongoing?",
    "How are global food prices being affected?"
]


#  "What are the sanctions imposed by NATO?",
#      "How has the energy market been influenced?",
#      "What humanitarian efforts are ongoing?",
#     "How are global food prices being affected?"

for query in test_queries:
    print(f"\n### Query: {query}")
    rag_query_verbose(query)



### Query: How are refugees impacted by the war in Ukraine?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


=== Context ===
 The UN says more than 12 million people have fled their homes since the Russian invasion of Ukraine. Russia is sending thousands more troops to Belarus. Is the country going to join the war in Ukraine? Civilians flee Ukraine's eastern city of Slovyansk and Kazakh oil is hit by a Russian court ruling. The most bombarded city in Ukraine’s war with Russia is key to Moscow’s military campaign. After withdrawing troops from the north of Ukraine, Russian shifts its focus to capturing more territory in the east. 
...

=== Generated Answer ===
 Civilians flee Ukraine's eastern city of Slovyansk and Kazakh oil is hit by a Russian court ruling

### Query: What humanitarian efforts are ongoing in Ukraine?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


=== Context ===
 Russia is sending thousands more troops to Belarus. Is the country going to join the war in Ukraine? Yevgeny says it is his responsibility as a Russian to help Ukrainians. The UN says more than 12 million people have fled their homes since the Russian invasion of Ukraine. Vladimir Putin has made it an annual event, but amid war in Ukraine 9 May has taken on new significance. The most bombarded city in Ukraine’s war with Russia is key to Moscow’s military campaign. 
...

=== Generated Answer ===
 Russia is sending thousands more troops to Belarus. Is the country going to join the war in Ukraine?

### Query: What actions are world leaders taking in response to the conflict?


...

=== Context ===
 European countries have taken different approaching to higher energy bills. A basic guide to how the economy is measured and why that calculation matters. What are the more radical options being discussed to protect people from fuel poverty this winter? People who rely on their cars for work tell Radio 1 Newsbeat of the impact of the petrol crisis. Record profits posted by energy company Shell and the build-up to the Coronation lead the papers. 
...

=== Generated Answer ===
 European countries have taken different approach to higher energy bills

### Query: What humanitarian efforts are ongoing?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


=== Context ===
 What are the more radical options being discussed to protect people from fuel poverty this winter? Billions promised in military aid as the PM restarts her agenda, following mourning for the Queen. They agree to allow safe passage for civilians and to protect relief workers but not to a ceasefire. Our correspondents analyse what impact additional funding might have in three key areas. UK concern over the conflict in the Middle East escalating even further makes headlines on Saturday.   
...

=== Generated Answer ===
 unanswerable

### Query: How are global food prices being affected?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


=== Context ===
 The rate at which prices are rising has fallen from record highs, but is still above the 2% target. The rate at which prices are rising has dropped from record highs, but remains above the 2% target. Sir John Major warns about rising prices, as a minister is criticised for advocating people working more hours. What are the more radical options being discussed to protect people from fuel poverty this winter? All-inclusive deals for some parts of Spain and Greece are more than 20% more expensive this year. 
...

=== Generated Answer ===
 All-inclusive deals for some parts of Spain and Greece are more than 20% more expensive this year
